# Tutorial: Ammonia Sensor Placement Optimization

This notebook is a production-grade, ready-to-run walkthrough for optimizing **chemical sensor network placement** around an ammonia release using `pyELDQM`.

It covers:
- scenario configuration (source, weather, AEGL thresholds, sensor parameters)
- Gaussian dispersion simulation and AEGL zone extraction
- sensor placement optimization using configurable strategy
- coverage metrics calculation (area, cost)
- interactive Folium map with sensor locations and detection rings displayed inline

## 1) Runtime Notes

This notebook supports two execution modes:
1. **Installed package mode** (`pip install pyeldqm`)
2. **Workspace mode** (running inside the project repository)

The import cell automatically detects and handles both.

In [12]:
from __future__ import annotations
import sys
from math import radians, cos, sin, asin, sqrt
from pathlib import Path
from datetime import datetime
import numpy as np
import folium
from IPython.display import display, Markdown, HTML

IMPORT_MODE = "package"
try:
    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.core.utils.sensor_optimization import SensorPlacementOptimizer
    from pyeldqm.core.visualization import (
        add_zone_polygons,
        ensure_layer_control,
        fit_map_to_polygons,
    )
except ModuleNotFoundError:
    IMPORT_MODE = "workspace"
    cwd = Path.cwd().resolve()
    candidates = [cwd] + list(cwd.parents)
    project_root = next(
        (p for p in candidates if (p / "core").exists() and (p / "app").exists()), None
    )
    if project_root is None:
        raise RuntimeError(
            "Could not resolve project root. Run from repository root or install pyeldqm via pip."
        )
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    from core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from core.meteorology.stability import get_stability_class
    from core.chemical_database import ChemicalDatabase
    from core.utils.features import setup_computational_grid
    from core.utils.zone_extraction import extract_zones
    from core.utils.sensor_optimization import SensorPlacementOptimizer
    from core.visualization import (
        add_zone_polygons,
        ensure_layer_control,
        fit_map_to_polygons,
    )

display(Markdown(f"**Import mode:** `{IMPORT_MODE}`"))

**Import mode:** `workspace`

## 2) Scenario Configuration

Update these values to model your own scenario.

In [13]:
# Chemical
CHEMICAL_NAME = "AMMONIA"
MOLECULAR_WEIGHT = 17.03              # g/mol
# Release location
SOURCE_LAT = 31.6911
SOURCE_LON = 74.0822
TIMEZONE_OFFSET_HRS = 5.0
# Release parameters
RELEASE_TYPE = "single"               # single | multi
SOURCE_TERM_MODE = "continuous"       # continuous | instantaneous
RELEASE_RATE = 800.0                  # g/s (continuous)
TANK_HEIGHT = 3.0                     # m
DURATION_MINUTES = 30.0               # min
MASS_RELEASED_KG = 500.0              # kg (instantaneous mode)
TERRAIN_ROUGHNESS = "URBAN"           # URBAN | RURAL
RECEPTOR_HEIGHT_M = 1.5               # m
# Weather
WEATHER_MODE = "manual"
WIND_SPEED = 2.5                      # m/s
WIND_DIRECTION = 90.0                 # degrees
TEMPERATURE_C = 25.0                  # C
HUMIDITY = 65.0                       # %
CLOUD_COVER = 30.0                    # %
# AEGL thresholds (ppm)
AEGL_THRESHOLDS = {
    "AEGL-1": 30.0,
    "AEGL-2": 160.0,
    "AEGL-3": 1100.0,
}
# Computational grid
X_MAX = 12000
Y_MAX = 4000
NX = 500
NY = 500
# Sensor placement parameters
SENSOR_STRATEGY = "boundary"          # boundary | grid | risk_weighted
SENSOR_NUM = 5                        # number of sensors to place
SENSOR_DETECTION_RANGE_M = 500.0      # detection radius per sensor (m)
SENSOR_MIN_SPACING_M = 200.0          # minimum distance between sensors (m)
SENSOR_COST_PER_SENSOR = 10000.0      # USD per sensor (for cost reporting)
SENSOR_POP_RASTER_PATH = ""           # optional: path to population raster

assert NX > 10 and NY > 10, "Grid resolution is too low."
assert X_MAX > 0 and Y_MAX > 0, "Grid extents must be positive."
assert WIND_SPEED > 0, "Wind speed must be > 0."
assert SENSOR_NUM > 0, "SENSOR_NUM must be at least 1."
print("Scenario configured successfully.")
print("All outputs will be shown inline in this notebook.")

Scenario configured successfully.
All outputs will be shown inline in this notebook.


## 3) Helper Functions

Distance utility for reporting maximum zone reach from the source.

In [14]:
def haversine_km(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * asin(sqrt(a)) * 6371.0


def max_distance_from_source_km(polygon, src_lat, src_lon):
    if polygon is None or polygon.is_empty:
        return None
    max_d = 0.0
    try:
        for lon, lat in polygon.exterior.coords:
            d = haversine_km(src_lat, src_lon, lat, lon)
            if d > max_d:
                max_d = d
    except Exception:
        return None
    return max_d if max_d > 0 else None

## 4) Chemical Properties

Fetch core records from the chemical database for transparency and traceability.

In [15]:
print("=" * 72)
print("CHEMICAL PROPERTIES")
print("=" * 72)
try:
    db = ChemicalDatabase()
    chem_data = next(
        (c for c in db.get_all_chemicals() if c.get("name") == CHEMICAL_NAME),
        None,
    )
    if chem_data:
        fields = [
            ("Name",                  "name"),
            ("Molecular Weight",      "molecular_weight"),
            ("Boiling Point (K)",     "boiling_point_K"),
            ("Density (kg/m3)",       "density_kgm3"),
            ("Vapor Pressure (kPa)",  "vapor_pressure_kPa"),
            ("IDLH (ppm)",            "idlh_ppm"),
            ("LC50 (ppm)",            "lc50_ppm"),
            ("AEGL-1 (60 min)",       "aegl1_60min"),
            ("AEGL-2 (60 min)",       "aegl2_60min"),
            ("AEGL-3 (60 min)",       "aegl3_60min"),
            ("ERPG-1",                "erpg1"),
            ("ERPG-2",                "erpg2"),
            ("ERPG-3",                "erpg3"),
        ]
        for label, key in fields:
            value = chem_data.get(key)
            if value not in (None, "", 0):
                print(f"{label:<26}: {value}")
    else:
        print(f"Name                     : {CHEMICAL_NAME}")
        print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
        print("Database record not found; using configured molecular weight.")
except Exception as ex:
    print(f"Name                     : {CHEMICAL_NAME}")
    print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
    print(f"Database query failed     : {ex}")

CHEMICAL PROPERTIES
Name                      : AMMONIA
Molecular Weight          : 17.03
AEGL-1 (60 min)           : 30 ppm
AEGL-2 (60 min)           : 160 ppm
AEGL-3 (60 min)           : 1100 ppm
ERPG-1                    : 25
ERPG-2                    : 150
ERPG-3                    : 750


## 5) Run Dispersion and Extract AEGL Zones

In [16]:
simulation_datetime = datetime.now()
weather = {
    "wind_speed":    WIND_SPEED,
    "wind_dir":      WIND_DIRECTION,
    "temperature_K": TEMPERATURE_C + 273.15,
    "humidity":      HUMIDITY / 100.0,
    "cloud_cover":   CLOUD_COVER / 100.0,
    "source":        "manual",
}
stability_class = get_stability_class(
    wind_speed=weather["wind_speed"],
    datetime_obj=simulation_datetime,
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    cloudiness_index=weather.get("cloud_cover", 0) * 10,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
)
X, Y, _, _ = setup_computational_grid(x_max=X_MAX, y_max=Y_MAX, nx=NX, ny=NY)
release_duration_s = DURATION_MINUTES * 60.0
source_q = RELEASE_RATE
dispersion_mode = "continuous" if SOURCE_TERM_MODE == "continuous" else "instantaneous"
concentration, U_local, resolved_stability, resolved_sources = calculate_gaussian_dispersion(
    weather=weather,
    X=X,
    Y=Y,
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    molecular_weight=MOLECULAR_WEIGHT,
    default_release_rate=source_q,
    default_height=TANK_HEIGHT,
    z_ref=3.0,
    z_measurement=RECEPTOR_HEIGHT_M,
    t=release_duration_s,
    t_r=release_duration_s,
    mode=dispersion_mode,
    sources=[{
        "lat": SOURCE_LAT, "lon": SOURCE_LON,
        "name": "Release Source",
        "height": TANK_HEIGHT, "rate": source_q,
        "color": "red",
    }],
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
    roughness=TERRAIN_ROUGHNESS,
    datetime_obj=simulation_datetime,
)
threat_zones = extract_zones(
    X, Y, concentration, AEGL_THRESHOLDS,
    SOURCE_LAT, SOURCE_LON, wind_dir=weather["wind_dir"],
)
print(f"Simulation time       : {simulation_datetime}")
print(f"Resolved wind speed   : {U_local:.2f} m/s")
print(f"Stability class       : {resolved_stability}")
print(f"Grid shape            : {concentration.shape}")

Simulation time       : 2026-02-23 21:58:27.796102
Resolved wind speed   : 2.50 m/s
Stability class       : F
Grid shape            : (500, 500)


## 6) Report Zone Distances

Maximum distance is measured from source to polygon boundary.

In [17]:
print("=" * 72)
print("METEOROLOGICAL CONDITIONS")
print("=" * 72)
print(f"Wind Speed            : {weather['wind_speed']:.2f} m/s")
print(f"Wind Direction        : {weather['wind_dir']:.0f} deg")
print(f"Temperature           : {weather['temperature_K'] - 273.15:.1f} C")
print(f"Humidity              : {weather['humidity'] * 100:.0f} %")
print(f"Cloud Cover           : {weather['cloud_cover'] * 100:.0f} %")
print(f"Stability Class       : {resolved_stability}")
print()
print("=" * 72)
print("AEGL THREAT ZONE DISTANCES FROM SOURCE")
print("=" * 72)
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    threshold = AEGL_THRESHOLDS.get(zone_name, "N/A")
    threshold_str = f"{threshold:.0f} ppm" if isinstance(threshold, (int, float)) else str(threshold)
    dist_km = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
    if dist_km is not None:
        print(
            f"{zone_name:<8} threshold: {threshold_str:<10} max distance: {dist_km:.3f} km ({dist_km * 1000:.0f} m)"
        )
    else:
        print(f"{zone_name:<8} threshold: {threshold_str:<10} max distance: -- (no zone formed)")

METEOROLOGICAL CONDITIONS
Wind Speed            : 2.50 m/s
Wind Direction        : 90 deg
Temperature           : 25.0 C
Humidity              : 65 %
Cloud Cover           : 30 %
Stability Class       : F

AEGL THREAT ZONE DISTANCES FROM SOURCE
AEGL-3   threshold: 1100 ppm   max distance: -- (no zone formed)
AEGL-2   threshold: 160 ppm    max distance: 0.622 km (622 m)
AEGL-1   threshold: 30 ppm     max distance: 1.769 km (1769 m)


## 7) Run Sensor Placement Optimization

The optimizer places sensors according to `SENSOR_STRATEGY`:
- **boundary** — sensors along the AEGL zone perimeters for early warning
- **grid** — uniform spatial coverage over the threat area
- **risk_weighted** — biased towards highest-concentration regions

An optional population raster (`SENSOR_POP_RASTER_PATH`) weights placement towards
high-density areas when provided.

In [18]:
population_engine = None
if SENSOR_POP_RASTER_PATH and str(SENSOR_POP_RASTER_PATH).strip():
    try:
        from core.population import SensorPopulationEngine
        population_engine = SensorPopulationEngine(str(SENSOR_POP_RASTER_PATH).strip())
        print(f"Population engine loaded : {SENSOR_POP_RASTER_PATH}")
    except Exception as pop_err:
        print(f"Population engine unavailable: {pop_err}")
else:
    print("Population engine        : not configured (placement by geometry only)")

optimizer = SensorPlacementOptimizer(
    population_engine=population_engine,
    config={
        "detection_range_m": SENSOR_DETECTION_RANGE_M,
        "min_sensor_spacing_m": SENSOR_MIN_SPACING_M,
        "cost_per_sensor": SENSOR_COST_PER_SENSOR,
    },
)

sensors = optimizer.optimize_sensor_placement(
    threat_zones,
    SOURCE_LAT,
    SOURCE_LON,
    num_sensors=SENSOR_NUM,
    strategy=SENSOR_STRATEGY,
    wind_direction=weather["wind_dir"],
)

metrics = optimizer.calculate_coverage_metrics(sensors, threat_zones)
coverage_area = metrics.get("coverage_area_km2", 0.0)
total_cost = SENSOR_COST_PER_SENSOR * len(sensors)

print(f"Strategy              : {SENSOR_STRATEGY}")
print(f"Sensors requested     : {SENSOR_NUM}")
print(f"Sensors deployed      : {len(sensors)}")
print(f"Detection range       : {SENSOR_DETECTION_RANGE_M:.0f} m")
print(f"Min sensor spacing    : {SENSOR_MIN_SPACING_M:.0f} m")
print(f"Coverage area         : {coverage_area:.2f} km2")
print(f"Estimated cost        : ${total_cost:,.0f}")

Population engine        : not configured (placement by geometry only)
Strategy              : boundary
Sensors requested     : 5
Sensors deployed      : 5
Detection range       : 500 m
Min sensor spacing    : 200 m
Coverage area         : 2.38 km2
Estimated cost        : $50,000


## 8) Report Sensor Positions

Tabulates each sensor's coordinates, priority, and distance from source.

In [19]:
print("=" * 72)
print("SENSOR NETWORK POSITIONS")
print("=" * 72)
print(f"{'#':<4} {'Latitude':>10} {'Longitude':>11} {'Priority':<12} {'Dist from source':>18}")
print("-" * 72)
for i, sensor in enumerate(sensors, start=1):
    slat = sensor.get("latitude", SOURCE_LAT)
    slon = sensor.get("longitude", SOURCE_LON)
    priority = sensor.get("priority", "medium").capitalize()
    dist_km = haversine_km(SOURCE_LAT, SOURCE_LON, slat, slon)
    print(f"{i:<4} {slat:>10.5f} {slon:>11.5f} {priority:<12} {dist_km:>14.3f} km")
print("=" * 72)

SENSOR NETWORK POSITIONS
#      Latitude   Longitude Priority       Dist from source
------------------------------------------------------------------------
1      31.69146    74.09819 High                  1.514 km
2      31.69156    74.09078 High                  0.813 km
3      31.69131    74.08337 High                  0.114 km
4      31.69068    74.08895 High                  0.641 km
5      31.69067    74.09637 High                  1.341 km


## 9) Build Sensor Placement Map (Inline)

In [20]:
m = folium.Map(location=[SOURCE_LAT, SOURCE_LON], zoom_start=13, tiles="OpenStreetMap")

# AEGL hazard zones
add_zone_polygons(
    m,
    {k: v for k, v in threat_zones.items() if k in ["AEGL-1", "AEGL-2", "AEGL-3"]},
    thresholds_context={"AEGL": AEGL_THRESHOLDS},
    name_prefix=None,
)

# Sensor network layer
sensor_fg = folium.FeatureGroup(name="Sensor Network", show=True)
for i, sensor in enumerate(sensors, start=1):
    slat = sensor.get("latitude", SOURCE_LAT)
    slon = sensor.get("longitude", SOURCE_LON)
    priority = sensor.get("priority", "medium").capitalize()

    # Sensor marker
    folium.CircleMarker(
        [slat, slon],
        radius=7,
        color="#E07000",
        fill=True,
        fill_color="#E07000",
        fill_opacity=0.85,
        tooltip=f"Sensor {i} | Priority: {priority}",
    ).add_to(sensor_fg)

    # Detection range ring
    folium.Circle(
        [slat, slon],
        radius=SENSOR_DETECTION_RANGE_M,
        color="#E07000",
        fill=True,
        fill_opacity=0.08,
        weight=1,
        tooltip=f"Sensor {i} detection range ({SENSOR_DETECTION_RANGE_M:.0f} m)",
    ).add_to(sensor_fg)

sensor_fg.add_to(m)

# Release source marker
folium.Marker([SOURCE_LAT, SOURCE_LON], tooltip="Release Source").add_to(m)

ensure_layer_control(m)
fit_map_to_polygons(m, threat_zones.values())
print("Sensor placement map generated. Displaying inline below.")

Sensor placement map generated. Displaying inline below.


## 10) Display Sensor Map Inline

In [21]:
if 'm' in globals() and m is not None:
    display(HTML(m._repr_html_()))
else:
    print("Map object not found. Run Cell 19 first.")

## 11) Inline Results Summary

Quick run summary for tutorial users.

In [22]:
peak_ppm = float(np.nanmax(concentration))
zone_stats = {}
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    zone_stats[zone_name] = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
print("Completed successfully.")
print(f"Chemical              : {CHEMICAL_NAME}")
print(f"Peak concentration    : {peak_ppm:.2f} ppm")
print(f"Strategy              : {SENSOR_STRATEGY}")
print(f"Sensors deployed      : {len(sensors)}")
print(f"Coverage area         : {coverage_area:.2f} km2")
print(f"Estimated cost        : ${total_cost:,.0f}")
print("Max zone distances:")
for z, d in zone_stats.items():
    if d is None:
        print(f"  {z}: no zone formed")
    else:
        print(f"  {z}: {d:.3f} km ({d * 1000:.0f} m)")

Completed successfully.
Chemical              : AMMONIA
Peak concentration    : 852.65 ppm
Strategy              : boundary
Sensors deployed      : 5
Coverage area         : 2.38 km2
Estimated cost        : $50,000
Max zone distances:
  AEGL-3: no zone formed
  AEGL-2: 0.622 km (622 m)
  AEGL-1: 1.769 km (1769 m)


## Troubleshooting

- If imports fail, run from repository root **or** install package mode: `pip install pyeldqm`.

- If the map does not render inline, rerun Cell 19 then Cell 21.

- If zones are empty, increase release rate/duration, reduce thresholds, or expand the grid extent.

- If fewer sensors are placed than `SENSOR_NUM`, increase grid extent or reduce `SENSOR_MIN_SPACING_M`.

- To weight placement towards populated areas, set `SENSOR_POP_RASTER_PATH` to a valid GeoTIFF raster path.